In [1]:
import theano
import theano.tensor as T
from theano.tensor.nnet import conv
from theano.tensor.signal import downsample
from theano import function
import numpy
import math
from theano.ifelse import ifelse
import time
import sys
import os

# theano.config.profile=True

In [2]:
# a = numpy.zeros((2*4,3*4,1,3))
# #print a, "\n\n"
# for i in range(4):
#     b = numpy.arange(i*18,18*(i+1)).reshape((2,3,1,3))
#     a[i*2:(i+1)*2,i*3:(i+1)*3] = b
# print a

In [3]:
def SharedZeros(shape):
    zeros = numpy.zeros(shape, dtype=theano.config.floatX)
    return theano.shared(zeros, borrow=True)

In [4]:
class ConvLayer1D(object):
    def __init__(self,rng, input, filter_shape, emb_size, activation=None, conv_border='full'):
        
        self.activation = activation
        out_fmaps, in_fmaps, height, width = filter_shape
    
        assert height == 1
        #
        # V2: 
        #
        # 
        self.input = input
        
        
#         fan_in = in_fmaps * 1 * width
#         fan_out = out_fmaps * 1 * width
#         W_bound = numpy.sqrt(6./(fan_in+fan_out))

        
        w = numpy.asarray(
                    numpy.random.normal(0, 0.05, 
                                size=(emb_size, out_fmaps, in_fmaps, 1, width)),
                    dtype=theano.config.floatX
                )
#         w = numpy.ones((emb_size, out_fmaps, in_fmaps, 1, width),
#                        dtype=theano.config.floatX)
        self.W = theano.shared(w, borrow=True)
        
        #
        # one conv for each time-line
        #
        conv_list = []
        for i in range(emb_size):
            # conv
            conv_out = conv.conv2d(
                input = self.input[:,:,i:i+1,:],
                filters=self.W[i],
                border_mode=conv_border
            )
            
            conv_list.append(conv_out)
        
        conv_out  = T.concatenate(conv_list, axis=2)
        

        #self.b = theano.shared(numpy.asarray(rng.rand(emb_size), 
        #                             dtype=theano.config.floatX))
        
        linear_output = conv_out #+ self.b.dimshuffle('x','x',0,'x')
        # use activation function
        if self.activation is None:
            self.output = linear_output
        else:
            self.output = self.activation(linear_output)
        
        # params
        self.params = [self.W]
        
        # regularization
        self.l1_reg = abs(self.W).sum()
        self.l2_reg = (self.W**2).sum()
        
        #### gradient history for adagrad
        self.Wgrad_hist = SharedZeros((emb_size, out_fmaps, in_fmaps, 1, width))
        #self.bgrad_hist = SharedZeros((emb_size,))
        
        self.grad_hist = [self.Wgrad_hist]
        
#         self.output = conv_out

In [5]:
# input = T.tensor4()
# rng=numpy.random
# convlayer = ConvLayer1D(rng, input, filter_shape=(2,2,1,2), 2)
# f = theano.function([input], convlayer.output)

# a = numpy.asarray(range(16), dtype=theano.config.floatX).reshape((2,2,2,2))
# a1 = f(a)

# print a.shape
# print a1.shape

# # kvec=T.ivector()
# # dkm = DynamicKMaxPoolLayer(input, kvec)
# # f1 = theano.function([input, kvec], dkm.output)

# # b = numpy.asarray([2, 3, 1, 4], dtype='int32')

# # print f1(a1,b).shape

In [6]:
# dynamic k-max polling
# pass a mask to take dynamic pooling into consideration
class DynamicKMaxPoolLayer(object):
    def __init__(self, input, kvec, kmax, mask=None):
        # input is a 4D tensor
        self.input = input
        
        if mask is None:
            self.indices = T.argsort(input, axis=3)
        else:
            #assert mask.shape[0] == input.shape[0]
            #assert mask.shape[1] == input.shape[2]
            
            #
            # advanced indexing: not work on GPU
            #
            
            # reshape to input shape
            indexes = mask.reshape((mask.shape[0], 1, 1, mask.shape[1]))
            # repeat along some axis
            indexes = T.extra_ops.repeat(indexes,input.shape[2], axis=2)
            indexes = T.extra_ops.repeat(indexes,input.shape[1], axis=1)
            # index zeros
            indexes = ( indexes< 1).nonzero()
            # set to -inf
            self.input = T.set_subtensor(self.input[indexes], float('-inf'))
            # sort
            self.indices = self.input.argsort(axis=3)
            # set to zero
            self.input = T.set_subtensor(self.input[indexes], float(0))
        
        #kmax = T.max(kvec)
        # for each k, use scan
        def step(input3d, indices3d, k, maxk):
            topk_indices = indices3d[:,:,-k:]
            ordered_topk_indices = topk_indices.sort(axis=2)
            reshaped = input3d.reshape((input3d.shape[0]*input3d.shape[1],input3d.shape[2]))
            output = reshaped[T.arange(reshaped.shape[0]).reshape((reshaped.shape[0],1)),
                              ordered_topk_indices.reshape((reshaped.shape[0],k))]
            output = output.reshape(ordered_topk_indices.shape)
            # padding zero 
            zeros = T.zeros((input3d.shape[0], input3d.shape[1], T.cast(maxk-k,dtype='int32')), 
                            dtype=theano.config.floatX)
            return T.concatenate([output, zeros], axis=2)
            
        results, _ = theano.scan(fn=step, 
                                 outputs_info=None,
                                 sequences=[self.input, self.indices, kvec], 
                                 non_sequences=[kmax])
        # output
        self.output = T.concatenate([results], axis=0)

In [7]:
# input = T.tensor4()
# kvec = T.ivector()
# mask = T.imatrix()
# dkm = DynamicKMaxPoolLayer(input, kvec, 4, mask=mask)

# f = theano.function([input, kvec, mask], dkm.output)

# a = numpy.arange(48, dtype=theano.config.floatX).reshape(2,2,3,4)
# b = numpy.asarray([3,1], dtype='int32')
# m = numpy.asarray([[1,1,1,0],[0,1,0,0]],dtype='int32')
# print a
# print f(a,b,m)

In [8]:
# k-max polling
# pass a mask to take dynamic pooling into consideration
class KMaxPoolLayer(object):
    def __init__(self, input, k, mask=None):
        # input is a 4D tensor
        self.input = input
        
        if mask is None:
            self.indices = T.argsort(input, axis=3)
        else:
            #assert mask.shape[0] == input.shape[0]
            #assert mask.shape[1] == input.shape[2]
            
            #
            # advanced indexing: not work on GPU !!!
            #
            
            # reshape to input shape
            indexes = mask.reshape((mask.shape[0], 1, 1, mask.shape[1]))
            # repeat along some axis
            indexes = T.extra_ops.repeat(indexes,input.shape[2], axis=2)
            indexes = T.extra_ops.repeat(indexes,input.shape[1], axis=1)
            # index zeros
            indexes = ( indexes< 1).nonzero()
            # set to -inf
            self.input = T.set_subtensor(self.input[indexes], float('-inf'))
            # sort
            self.indices = self.input.argsort(axis=3)
            # set to zero
            self.input = T.set_subtensor(self.input[indexes], float(0))
            
        # topk indices
        topk_indices = self.indices[:,:,:,-k:]
        # re-store the order
        ordered_topk_indices = topk_indices.sort(axis=3)
        
        # this is one implemetation by reshape
        # should consider more efficient ways
        reshaped = self.input.reshape((T.prod(self.input.shape[:3]),self.input.shape[3]))
        output = reshaped[T.arange(reshaped.shape[0]).reshape((reshaped.shape[0],1)),
                          ordered_topk_indices.reshape((reshaped.shape[0],k))]
        self.output = output.reshape(ordered_topk_indices.shape)
#         self.oti = ordered_topk_indices
#         self.ti = topk_indices
#         self.get_indices = function([input, k], self.oti)

In [9]:
# input = T.tensor4()
# mask = T.imatrix()

# dklayer = KMaxPoolLayer(input, 3, mask)
# f = theano.function([input, mask], dklayer.output)

# a = numpy.asarray(range(32), dtype=theano.config.floatX).reshape((2,2,2,4))
# # numpy.random.shuffle(a)
# b = numpy.asarray([1,1,0,0,1,0,1,0], dtype='int32').reshape(2,4)

# out = f(a,b)
# print a
# print out

In [10]:
class FoldingLayer(object):
    def __init__(self, input, fold_len=2):
        shape = input.shape
        extra = shape[2]%fold_len
        input = ifelse(T.eq(extra, 0), 
                       input, 
                       T.concatenate([input, 
                                     T.zeros((shape[0], shape[1], fold_len-extra, shape[3]),
                                             dtype=theano.config.floatX)],
                                     axis=2))
        
        new_len = T.cast(T.ceil(shape[2]*1.0/fold_len), 'int32')
        X = input.reshape((shape[0], shape[1], new_len, fold_len, shape[3]))
        X = T.sum(X, axis=3)
        X = X.reshape((shape[0],shape[1],new_len,shape[3]))
        self.output = X

In [11]:
# a = numpy.asarray(range(24), dtype=theano.config.floatX).reshape((2,2,3,2))
# input = T.tensor4()
# fold = FoldingLayer(input)
# f = theano.function([input],fold.output)
# print a
# print f(a)

In [12]:
class NonLinearLayer(object):
    def __init__(self, rng, input, size, fmaps,  activation=T.tanh):
        self.activation = activation
        self.b = theano.shared(numpy.asarray(numpy.random.rand(fmaps, size), dtype=theano.config.floatX))
        self.output = self.activation(input + self.b.dimshuffle('x',0,1,'x'))
        self.params = [self.b]

        self.bgrad_hist = SharedZeros((fmaps, size))
        self.grad_hist = [self.bgrad_hist]

In [13]:
# a = numpy.asarray(range(32), dtype=theano.config.floatX).reshape((2,2,4,2))
# input = T.tensor4()
# nl = NonLinearLayer(input, 4)
# f = theano.function([input],nl.output)
# print a
# print f(a)

In [14]:
class DropoutLayer(object):
    def __init__(self, rng, input, p):
        """p is the probablity of dropping a unit
        """
        srng = theano.tensor.shared_randomstreams.RandomStreams(numpy.random.randint(999999))
        # p=1-p because 1's indicate keep and p is prob of dropping
        mask = srng.binomial(n=1, p=1-p, size=input.shape)
        # The cast is important because
        # int * float32 = float64 which pulls things off the gpu
        self.output = input * T.cast(mask, theano.config.floatX)

In [15]:
# a = numpy.asarray(range(32), dtype=theano.config.floatX).reshape((2,2,4,2))
# input = T.tensor4()
# drop = DropoutLayer(None, input, 0.5)
# f = theano.function([input],drop.output)
# print a
# print f(a)

In [16]:
# TODO: input reshape
# TODO: define error, params, and functions
class ConvSentModel (object):
    # so many parameters !!!
    def __init__(self, rng, input, sentlen, mask, vocab_size=1000, num_feats=48, max_len=58, 
                 embedding=None, m1=7, m2=5, fmap1=6, fmap2=14, topk=4, 
                 num_class=2, drop_prob=0.5, fold_len=2):
        
        ##rng = numpy.random.RandomState(1234)
        
        self.input = input
        self.sentlen = sentlen
        self.mask = mask
        
        # input variable
        #input = T.imatrix()
        #sentlen = T.ivector()
        
        # get batch size
        batch_size = self.input.shape[0]
        
        # wordvec parameter
        if embedding is None:
            w_bound = numpy.sqrt(6./(vocab_size+num_feats))
            self.embedding = theano.shared(numpy.asarray(
                    rng.normal(0, 0.05,
                               size=(num_feats,vocab_size)), 
                    dtype=theano.config.floatX))
        else:
            self.embedding = embedding
        # add an extrac row at last for padding short sentences
        emb_ext = T.concatenate([self.embedding, T.zeros((num_feats,1), 
                                                         dtype=theano.config.floatX)], 
                                axis=1)
        
        # get input matrix for batched sentences
        # resshape the matrix for 1D convolution layer input
        # (batch_size, input_feature_maps, num_feats, max_sent_len)
        in_matrix = emb_ext.transpose()[self.input].reshape(
            (batch_size, 1, max_len, num_feats)).dimshuffle(0,1,3,2)
        
        
        ###
        ### the first part is : wide_conv + folding + dynamic k-max + non-linear
        ###
        
        # wide convolution
        #conv1_output_shape = (batch_size, fmap1, num_feat, max_len+m1-1)
        conv1 = ConvLayer1D(rng, in_matrix, 
                            filter_shape=(fmap1, 1, 1, m1),
                            emb_size=num_feats,
                            activation=None)
        # folding
        fold1 = FoldingLayer(conv1.output, fold_len=fold_len)
        
        # now the num_feats has become to math.ceil(num_features/2)
        num_feats1 = num_feats/fold_len
#         num_feats1 = num_feats
        # dynamic k-max
        # depends on the length of input and it's depth in the network
        # the first dynamic-kmax-pooling
        # zero are padded if < max-k
        max_dk = int(math.ceil(0.5 * max_len))
        total_len1 = max_len+m1-1
        # a mask is need to enable variable sent-length
        # define mask-step which create a mask-vec for one sent
#         def maskstep1(slen):
#             mask_vec = T.zeros((total_len1,))
#             mask_vec = T.set_subtensor(mask_vec[:slen+m1-1], 1)
#             return mask_vec.transpose(), T.cast(T.max([T.ceil(0.5 * slen), topk]), dtype='int32')
#         # the max_len after convolution
        
#         [mask_vecs1, dkvec1], _ = theano.scan(fn=maskstep1, 
#                                  outputs_info=None,
#                                  sequences=[self.sentlen])
# #         concatenate together to get a matrix
#         mask1 = T.concatenate([mask_vecs1], axis=0)
        # create a dkmax-pool layer
        real_dkvec1 = T.cast(T.ceil(0.5 * self.sentlen), dtype='int32')
        dkvec1 = T.set_subtensor(real_dkvec1[(real_dkvec1<topk).nonzero()], topk)
        mask1 = self.mask[:, :total_len1]
        dkmax = DynamicKMaxPoolLayer(fold1.output, dkvec1, max_dk, mask=mask1)
        
        # apply a non-linear function
        nonlinear1 = NonLinearLayer(rng, dkmax.output, num_feats1, fmap1)
        
        ###
        ### second part is : wide_conv + folding + k-max + non-linear
        ###
        
        # wide convolution
        conv2 = ConvLayer1D(rng, nonlinear1.output, 
                            filter_shape=(fmap2, fmap1, 1, m2), 
                            emb_size=num_feats1,
                            activation=None)
        # folding
        fold2 = FoldingLayer(conv2.output, fold_len=fold_len)
        num_feats2 = num_feats1/fold_len
        
        # k-max
        # also need a mask
        total_len2 = max_dk + m2 - 1
#         def maskstep2(real_dk):
#             mask_vec = T.zeros((total_len2,))
#             mask_vec = T.set_subtensor(mask_vec[:real_dk+m2-1], 1)
#             return mask_vec.transpose()
        
#         mask_vecs2, _ = theano.scan(fn=maskstep2, 
#                                     outputs_info=None,
#                                     sequences=[dkvec1])
#         mask2 = T.concatenate([mask_vecs2], axis=0)
        # create a kmaxlayer
        mask2 = self.mask[:, total_len1:]
        kmax = KMaxPoolLayer(fold2.output, topk, mask=mask2)
        
        #apply another non-linear
        nonlinear2 = NonLinearLayer(rng, kmax.output, num_feats2, fmap2)
        
        # flattern the output to get sentence vec
        sent_dim = topk * num_feats2 * fmap2
        sentvecs = nonlinear2.output.reshape((batch_size,sent_dim))
        
        ###
        ### before the last part: apply a dropoutlayer
        ###
        
        # not used now
        dropout = DropoutLayer(rng, sentvecs, drop_prob)
        
        ###
        ### the third part, also the last part, is a fully connected layer
        ###
        
        w_bound = numpy.sqrt(6./(num_class+sent_dim))
        self.W_out = theano.shared(
            numpy.asarray(
                numpy.random.normal(0, 0.05,
                           size=(sent_dim,num_class)),
                dtype=theano.config.floatX))
        self.b_out = theano.shared(numpy.asarray(
                numpy.random.rand(num_class), 
                dtype=theano.config.floatX))
        
        #softmax output
        self.p_y_given_x = T.nnet.softmax(T.dot(dropout.output, self.W_out) + self.b_out)
        self.y_pred = T.argmax(self.p_y_given_x, axis=1)
        
        #softmax output for test, without dropout
        self.p_y_given_x_test = T.nnet.softmax(T.dot(sentvecs, self.W_out) + self.b_out)
        self.y_pred_test = T.argmax(self.p_y_given_x_test, axis=1)
        
        
        # some useful attributs
        self.params = ( conv1.params + nonlinear1.params 
                       + conv2.params + nonlinear2.params 
                       + [self.embedding, self.W_out, self.b_out]
                       )
        
        self.l1_reg = (conv1.l1_reg + conv2.l1_reg 
                       + abs(self.W_out).sum() 
                       + abs(self.embedding).sum())
        self.l2_reg = (conv1.l2_reg + conv2.l2_reg 
                       + (self.W_out ** 2).sum() 
                       + (self.embedding**2).sum())
        
        # gradient history for adagrad
        self.embgrad_hist = SharedZeros((num_feats,vocab_size))
        self.Wgrad_hist = SharedZeros((sent_dim,num_class))
        self.bgrad_hist = SharedZeros((num_class,))
        # the same order with params
        self.grad_hist = (conv1.grad_hist + nonlinear1.grad_hist
                         + conv2.grad_hist + nonlinear2.grad_hist
                         + [self.embgrad_hist, self.Wgrad_hist, self.bgrad_hist])
    
    def negative_log_likelihood(self,y):
        return -T.mean(T.log(self.p_y_given_x)[T.arange(y.shape[0]), y])
    
    def errors(self,y):
        # check if y has same dimension of y_pred
        if y.ndim != self.y_pred_test.ndim:
            raise TypeError(
                'y should have the same shape as self.y_pred',
                ('y', y.type, 'y_pred', self.y_pred_test.type)
            )
        # check if y is of the correct datatype
        if y.dtype.startswith('int'):
            # the T.neq operator returns a vector of 0s and 1s, where 1
            # represents a mistake in prediction
            return T.mean(T.neq(self.y_pred_test, y))
        else:
            raise NotImplementedError()
            
#         # gradients
#         gparams = T.grad(self.cost, self.params)
#         updates = []
#         for param, gparam in zip(self.params, gparams):
#             updates.append([param, param - learning_rate * gparam])
        
#         # tools: some useful functions
#         self.Train = theano.function([input, sentlen, y], self.cost, updates=updates)
#         self.SentenceVector = theano.function([input, sentlen], sentvecs)
#         self.Predict = theano.function([input, sentlen], self.y_pred)
        

In [17]:
# ConvSentModel(100, 10, 20)
# rng = numpy.random.RandomState(1234)
# x = T.imatrix('x')  # the data is presented as rasterized images
# s = T.ivector('s') # sentence length
# ConvSentModel(rng, x,s)

In [18]:
# mask = T.zeros((3,5))
# sent = ivector()
# mask1 = T.set_subtensor(mask[T.arange(sent.shape[1]), ], 1)
# f = theano.function([sent], mask1)
# m = f([1,3,4])
# print m

In [31]:
###
### read data to test
###

def load_sst_data(emb_size=48):
    ''' Loads the dataset

    :type dataset: string
    :param dataset: the path to the dataset (here MNIST)
    '''

    #############
    # LOAD DATA #
    #############
    matFile = 'sst/SST_binary.mat'
    import scipy.io
    data = scipy.io.loadmat(matFile)
    
#     print '... loading data'

#     # Load the dataset
#     # keys: 
#     data = scipy.io.loadmat(matFile)
    
#     pklFile = 'sst/SST_dep.pkl'
#     import cPickle
#     data = cPickle.load(open(pklFile, 'rb'))
    
    train_set = data['train']
    train_lbl_set = data['train_lbl']
    valid_set = data['valid']
    valid_lbl_set = data['valid_lbl']
    test_set = data['test']
    test_lbl_set = data['test_lbl']
    emb = data['vocab_emb'][:emb_size]
    
    embedding = theano.shared(numpy.asarray(emb, 
                                            dtype=theano.config.floatX),
                             borrow=True)
    
    #train_set, valid_set, test_set format: tuple(input, target)
    #input is an numpy.ndarray of 2 dimensions (a matrix)
    #witch row's correspond to an example. target is a
    #numpy.ndarray of 1 dimensions (vector)) that have the same length as
    #the number of rows in the input. It should give the target
    #target to the example with the same index in the input.

    def shared_dataset(data_x, data_ys, borrow=True):
        """ Function that loads the dataset into shared variables

        The reason we store our dataset in shared variables is to allow
        Theano to copy it into the GPU memory (when code is run on GPU).
        Since copying data into the GPU is slow, copying a minibatch everytime
        is needed (the default behaviour if the data is not in a shared
        variable) would lead to a large decrease in performance.
        """
        data_y, data_s = data_ys[:,0], data_ys[:,1]
        shared_x = theano.shared(numpy.asarray(data_x-1,
                                               dtype='int32'),
                                 borrow=borrow)
        shared_y = theano.shared(numpy.asarray(data_y-1,
                                               dtype='int32'),
                                 borrow=borrow)
        shared_s = theano.shared(numpy.asarray(data_s,
                                               dtype='int32'),
                                 borrow=borrow)
        # When storing data on the GPU it has to be stored as floats
        # therefore we will store the labels as ``floatX`` as well
        # (``shared_y`` does exactly that). But during our computations
        # we need them as ints (we use labels as index, and if they are
        # floats it doesn't make sense) therefore instead of returning
        # ``shared_y`` we will have to cast it to int. This little hack
        # lets ous get around this issue
        return shared_x, shared_y, shared_s

    test_set_x, test_set_y, test_set_s = shared_dataset(test_set, test_lbl_set)
    valid_set_x, valid_set_y, valid_set_s = shared_dataset(valid_set, valid_lbl_set)
    train_set_x, train_set_y, train_set_s = shared_dataset(train_set, train_lbl_set)

    rval = [(train_set_x, train_set_y, train_set_s), 
            (valid_set_x, valid_set_y, valid_set_s),
            (test_set_x, test_set_y, test_set_s), embedding]
    return rval

In [32]:
# matFile = 'sst/SST_binary.mat'
# import scipy.io
# data = scipy.io.loadmat(matFile)

# print len(data['index'])
# #for i in range(len(data['index'])):
# print data['index'][65]
# x = [data['index'][i,0] for i in xrange(len(data['index']))]
# y = [i[0] if len(i) > 0 else '' for i in x]
# print [i for i in y]
# print '... loading data'

# data = scipy.io.loadmat(matFile)

# print sum((data['train_lbl'][:,0]-1)[:1000])

In [33]:
'''
two masks: one for dkmax, one for kmax
concatenated
'''

def create_mask(S, max_len, topk, m1, m2):
    size = S.shape[0]
    l1 = max_len+m1-1
    max_dk = int(math.ceil(max_len*0.5))
    l2 = max_dk + m2-1
    mask = numpy.zeros((size, l1+l2),
                      dtype='int32')
    for i in range(size):
        real_m1 = S[i] + m1-1
        mask[i, :real_m1] = 1
        real_dk = max([topk, int(math.ceil(0.5*S[i]))])
        real_m2 = real_dk + m2-1
        mask[i, l1:l1+real_m2] = 1
    M = theano.shared(mask, borrow=True)
    return M

In [34]:
def test_DCNN(batch_size = 5, l1_rate=0, l2_rate=0.0001, learning_rate=0.1,
             n_epochs=20):
    """
    Demonstrate stochastic gradient descent optimization for a multilayer
    perceptron

    This is demonstrated on MNIST.

    :type learning_rate: float
    :param learning_rate: learning rate used (factor for the stochastic
    gradient

    :type L1_reg: float
    :param L1_reg: L1-norm's weight when added to the cost (see
    regularization)

    :type L2_reg: float
    :param L2_reg: L2-norm's weight when added to the cost (see
    regularization)

    :type n_epochs: int
    :param n_epochs: maximal number of epochs to run the optimizer

    :type dataset: string
    :param dataset: the path of the MNIST dataset file from
                 http://www.iro.umontreal.ca/~lisa/deep/data/mnist/mnist.pkl.gz


   """
    
    emb_size = 8
    
    datasets = load_sst_data(emb_size)

    train_set_x, train_set_y, train_set_s = datasets[0]
    valid_set_x, valid_set_y, valid_set_s = datasets[1]
    test_set_x, test_set_y, test_set_s = datasets[2]
    embedding = datasets[3]
    #vocab_size = datasets[4]
    #sent_length = datasets[5]
    
    #emb_size = 48 #embedding.get_value(borrow=True).shape[0]
    vocab_size = embedding.get_value(borrow=True).shape[1]
    sent_length = train_set_x.get_value(borrow=True).shape[1]
    
    print 'emb_size: ', emb_size
    print 'vocab_size: ', vocab_size
    print 'max_sent_len: ', sent_length
    
    
    # create mask
    m1=7
    m2=5
    topk=4
    
    train_set_m = create_mask(train_set_s.get_value(borrow=True),
                             sent_length, topk, m1, m2)
    valid_set_m = create_mask(valid_set_s.get_value(borrow=True),
                             sent_length, topk, m1, m2)
    test_set_m = create_mask(test_set_s.get_value(borrow=True),
                             sent_length, topk, m1, m2)
    
#     print train_set_m.get_value(borrow=True)
    # end crete mask
    
    print (train_set_x.get_value(borrow=True).shape[0],
           valid_set_x.get_value(borrow=True).shape[0],
           test_set_x.get_value(borrow=True).shape[0])

    # compute number of minibatches for training, validation and testing
    n_train_batches = train_set_x.get_value(borrow=True).shape[0] / batch_size
    n_valid_batches = valid_set_x.get_value(borrow=True).shape[0] / batch_size
    n_test_batches = test_set_x.get_value(borrow=True).shape[0] / batch_size

    ######################
    # BUILD ACTUAL MODEL #
    ######################
    print '... building the model'

    # allocate symbolic variables for the data
    index = T.lscalar()  # index to a [mini]batch
    x = T.imatrix('x')  # the data is presented as rasterized images
    s = T.ivector('s') # sentence length
    y = T.ivector('y')  # the labels are presented as 1D vector of
                        # [int] labels
    m = T.imatrix('m') # mask

    rng = numpy.random.RandomState(123456789)

    # construct the MLP class
    classifier = ConvSentModel(
        rng=rng,
        input=x,
        sentlen=s,
        mask=m,
        vocab_size = vocab_size,
        num_feats=emb_size,
        max_len=sent_length,
        embedding=embedding,
        topk=topk, m1=m1, m2=m2
    )

    # start-snippet-4
    # the cost we minimize during training is the negative log likelihood of
    # the model plus the regularization terms (L1 and L2); cost is expressed
    # here symbolically
    
    cost = (
        classifier.negative_log_likelihood(y)
        + l1_rate * classifier.l1_reg
        + l2_rate * classifier.l2_reg
    )
    # end-snippet-4

    # compiling a Theano function that computes the mistakes that are made
    # by the model on a minibatch
    test_model = theano.function(
        on_unused_input='warn',
        inputs=[index],
        outputs=classifier.errors(y),
        givens={
            x: test_set_x[index:],
            s: test_set_s[index:],
            y: test_set_y[index:],
            m: test_set_m[index:]
        },
        mode='FAST_RUN'
    )

    validate_model = theano.function(
        on_unused_input='warn',
        inputs=[index],
        outputs=classifier.errors(y),
        givens={
            x: valid_set_x[index:],
            s: valid_set_s[index:],
            y: valid_set_y[index:],
            m: valid_set_m[index:]
        },
        mode='FAST_RUN'
    )
    
    # start-snippet-5
    # compute the gradient of cost with respect to theta (sotred in params)
    # the resulting gradients will be stored in a list gparams
    gparams = [T.grad(cost, param) for param in classifier.params]

    # specify how to update the parameters of the model as a list of
    # (variable, update expression) pairs

    # given two list the zip A = [a1, a2, a3, a4] and B = [b1, b2, b3, b4] of
    # same length, zip generates a list C of same size, where each element
    # is a pair formed from the two lists :
    #    C = [(a1, b1), (a2, b2), (a3, b3), (a4, b4)]
    
#     updates = [
#         (param, param - sq * gparam)
#         for param, gparam in zip(classifier.params, gparams)
#     ]
    
    # adagrad update
    updates = []
    for param, gparam, grad_hist in zip(classifier.params, gparams, classifier.grad_hist):
        gh = grad_hist + gparam ** 2
        sq = T.sqrt(gh)
        sq = T.set_subtensor(sq[sq.nonzero()], learning_rate / sq[sq.nonzero()])
        
        updates.append((param, param - sq * gparam))
        updates.append((grad_hist, grad_hist + gparam ** 2))
    
#     updates.append((grad_hist, grad_hist + gparams.flatten() ** 2))

    # compiling a Theano function `train_model` that returns the cost, but
    # in the same time updates the parameter of the model based on the rules
    # defined in `updates`
    train_model = theano.function(
        on_unused_input='warn',
        inputs=[index],
        outputs=cost,
        updates=updates,
        givens={
            x: train_set_x[index * batch_size: (index + 1) * batch_size],
            s: train_set_s[index * batch_size: (index + 1) * batch_size],
            y: train_set_y[index * batch_size: (index + 1) * batch_size],
            m: train_set_m[index * batch_size: (index + 1) * batch_size]
        },
        mode='FAST_RUN'
    )
    # end-snippet-5
    
    # reset grad_hist
    xx = T.scalar()
    update_reset = [(grad_hist, T.set_subtensor(grad_hist[:], xx)) for grad_hist in classifier.grad_hist]
    reset_hist = theano.function([xx],updates=update_reset)

    ###############
    # TRAIN MODEL #
    ###############
    print '... training'

    # early-stopping parameters
    patience = 10000  # look as this many examples regardless
    patience_increase = 2  # wait this much longer when a new best is
                           # found
    improvement_threshold = 0.995  # a relative improvement of this much is
                                   # considered significant
    validation_frequency = 10#min(n_train_batches, patience / 2)
                                  # go through this many
                                  # minibatche before checking the network
                                  # on the validation set; in this case we
                                  # check every epoch

    best_validation_loss = numpy.inf
    best_iter = 0
    test_score = 0.
    start_time = time.clock()

    epoch = 0
    done_looping = False

    while (epoch < n_epochs) and (not done_looping):
        epoch = epoch + 1
        reset_hist(0.)
        for minibatch_index in xrange(n_train_batches):

            minibatch_avg_cost = train_model(minibatch_index)
            
            #print classifier.grad_hist[-2].get_value(borrow=True)[0,:5]
            #print "gp_weig : ", classifier.params[-1].get_value(borrow=True)
            #print "gp_valu : ", gp
            #print "gp_hist : ", classifier.grad_hist[-1].get_value(borrow=True)
            
            # iteration number
            iter = (epoch - 1) * n_train_batches + minibatch_index

            #if (iter + 1) % validation_frequency == 0:
            if   (minibatch_index+1) %  validation_frequency == 0 or minibatch_index+1 == n_train_batches:
                # compute zero-one loss on validation set
                validation_losses = validate_model(0)
                this_validation_loss = validation_losses

                print(
                    'epoch %i, minibatch %i/%i, PPL %f, validation error %f %%' %
                    (
                        epoch,
                        minibatch_index + 1,
                        n_train_batches,
                        math.exp(minibatch_avg_cost),
                        this_validation_loss * 100.
                    )
                )
                
                total_size = 0
                total_value = 0.
                for param in classifier.params:
                    value = param.get_value(borrow=True)
                    total_size += numpy.prod(value.shape)
                    total_value += abs(value).sum()
                avgw = total_value/total_size
                print "average weights %f" % (avgw)

                # if we got the best validation score until now
                if this_validation_loss < best_validation_loss:
                    #improve patience if loss improvement is good enough
                    if (
                        this_validation_loss < best_validation_loss *
                        improvement_threshold
                    ):
                        patience = max(patience, iter * patience_increase)

                    best_validation_loss = this_validation_loss
                    best_iter = iter

                    # test it on the test set
                    test_losses = test_model(0)
                    test_score = test_losses

                    print(('     epoch %i, minibatch %i/%i, test error of '
                           'best model %f %%') %
                          (epoch, minibatch_index + 1, n_train_batches,
                           test_score * 100.))

#             if patience <= iter:
#                 done_looping = True
#                 break

    end_time = time.clock()
    print(('Optimization complete. Best validation score of %f %% '
           'obtained at iteration %i, with test performance %f %%') %
          (best_validation_loss * 100., best_iter + 1, test_score * 100.))
    print >> sys.stderr, ('The code ran for %.2fm' % ((end_time - start_time) / 60.))

In [35]:
# a = numpy.arange(20).reshape(4,5)
# b = numpy.asarray([[3,4],[1,2]])
# print a
# print a.transpose()[b].transpose([0,2,1]).reshape((2,1,4,2))
# print a.transpose()[b].reshape((2,1,2,4)).transpose([0,1,3,2])

In [ ]:
#theano.config.exception_verbosity='high'
#theano.config.optimizer='None'
if __name__ == '__main__':
    test_DCNN(batch_size = 2, n_epochs=1)

emb_size:  8
vocab_size:  15447
max_sent_len:  58
(100, 872, 1821)